In [ ]:
import glob
import os
import os.path as osp
from typing import Tuple, Any

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from notebook_lib import *

In [ ]:
ds_results_dir = '../domain_shift_results'
tableshift_results_dir = '../ray_train_results'

In [ ]:
ds_results = read_domain_shift_results(ds_results_dir)

In [ ]:
# TODO(jpgard): read in the rest of the results files, align/combine them, and visualize the results per-dataset.
# Goal is to have one scatter plot per dataset, with model types as markers.